<a href="https://www.kaggle.com/code/kparthiban007/time-series-analysis?scriptVersionId=212076842" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="background-color:#FFEBCD; color:#FF8C00; font-size:25px; padding:10px; text-align:center; border-radius:25px;">
    Time Series Analysis
</div>

In [1]:
#Importing Necessary Libararies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
# loading Data as a Pandas Dataframe
train_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv")

train_df.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [2]:
test_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
test_df.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [3]:
oil_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/oil.csv")
oil_df.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [4]:
holiday_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv")
holiday_df.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [5]:
holiday_df['type'].value_counts()

type
Holiday       221
Event          56
Additional     51
Transfer       12
Bridge          5
Work Day        5
Name: count, dtype: int64

In [6]:
holiday_df['locale'].value_counts()

locale
National    174
Local       152
Regional     24
Name: count, dtype: int64

In [7]:
transaction_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")
transaction_df.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [8]:
stores_df = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/stores.csv")
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [9]:
stores_df['city'].value_counts()

city
Quito            18
Guayaquil         8
Cuenca            3
Santo Domingo     3
Manta             2
Latacunga         2
Machala           2
Ambato            2
Quevedo           1
Esmeraldas        1
Loja              1
Libertad          1
Playas            1
Daule             1
Babahoyo          1
Salinas           1
Puyo              1
Guaranda          1
Ibarra            1
Riobamba          1
Cayambe           1
El Carmen         1
Name: count, dtype: int64

In [10]:
print(f"Shape of the Data : {train_df.shape}")

Shape of the Data : (3000888, 6)


In [11]:
train_df.info() # information About the Data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


In [12]:
train_df.isnull().sum() # checking for null values

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [13]:
train_df.duplicated().sum() # checking for duplicate values

0

In [14]:
train_df['date'] =pd.to_datetime(train_df['date']) # converting object datatype into time datatype

In [15]:
categoric_columns = train_df.select_dtypes(include='object').columns
numeric_columns = train_df.select_dtypes(include=['int64','float64']).drop(columns ='id')
numeric_columns =numeric_columns.columns
categoric_columns ,numeric_columns

(Index(['family'], dtype='object'),
 Index(['store_nbr', 'sales', 'onpromotion'], dtype='object'))

In [16]:
train_df['sales'].value_counts()

sales
0.000       939130
1.000       115291
2.000        85959
3.000        68575
4.000        57846
             ...  
116.541          1
363.533          1
141.322          1
409.879          1
2419.729         1
Name: count, Length: 379610, dtype: int64

In [17]:
#sns.histplot(x=train_df['sales'])